In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load the dataset
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'text'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'text'])

# Convert labels to binary (ham = 0, spam = 1)
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})

# Display dataset info
train_df.head()


In [ ]:
# prompt: Using dataframe train_df: box plot

import seaborn as sns
import matplotlib.pyplot as plt
# Create the box plot
sns.boxplot(x='label', y='text', data=train_df) # Assuming 'text' can be numerically represented for plotting
plt.show()


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Create and train the vectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(train_df['text'])

# Transform the training data
training_data = vectorizer.transform(train_df['text'])


# Create and train the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(training_data.shape[1],)), # Input layer with correct shape
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(vectorizer.transform(train_df['text']).toarray(), train_df['label'], epochs=10)

In [ ]:
# %%
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    # Transform input text
    transformed_text = vectorizer.transform([pred_text])
    # Get model prediction
    prob = model.predict(transformed_text.toarray())[0][0]
    # Determine label
    label = "spam" if prob > 0.5 else "ham"
    return [prob, label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# %%
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
